In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

C:\Users\lenovo\.virtualenvs\llmzoomcamp-nAbKfTAh\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
embedding_vector = embedding_model.encode(user_question)

In [4]:
embedding_vector[0]

0.07822264

In [5]:
#1 0.07

In [6]:
import requests 
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
documents = [doc for doc in documents if "machine-learning-zoomcamp" in doc['course']]
print(f"Number of documents: {len(documents)}")

Number of documents: 375


In [9]:
import numpy as np
model = embedding_model

embeddings = []

for doc in documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

In [10]:
print(f"Shape of X: {X.shape}")

Shape of X: (375, 768)


In [ ]:
#2 (375, 768)

In [15]:
scores = X.dot(embedding_vector)

In [16]:
highest_score = scores.max()
print(f"The highest score is: {highest_score}")

The highest score is: 0.6506574153900146


In [ ]:
#3 0.6506574153900146

In [19]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(embedding_vector)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(embedding_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [20]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [39]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(embedding_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [42]:
len(ground_truth)

1830

In [56]:
l = [ doc['document'] for doc in ground_truth]

In [70]:
[result['id'] in l for result in results]

[True, True, True, True, True]

In [61]:
len(set(l))

374

In [68]:
hits = 0
#total = len(set(l))
total = len(l)
for item in ground_truth:
    query = item['question']
    expected_id = item['document']
    # Search
    results = search_engine.search(embedding_vector, num_results=5)
    
    # Check if the expected document is in the results
    if any(result['id'] == expected_id for result in results):
        hits += 1

print(f"Hit-rate: {( hits / 5):.2f}")

Hit-rate: 5.00


In [ ]:
#4 100

In [74]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

index_name = "ml_zoomcamp"

settings = {
    "mappings": {
        "properties": {
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,  # Modifié pour correspondre à la dimension de nos embeddings
                "index": True,
                "similarity": "cosine"
            },
            "text": {"type": "text"}
        }
    }
}

es.indices.create(index=index_name, body=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml_zoomcamp'})

In [75]:
for i, doc in enumerate(documents):
    es.index(index=index_name, id=doc['id'], body={
        "text": doc['text'],
        "text_vector": X[i].tolist()  # Convertir en liste pour JSON
    })

In [76]:
query = "I just discovered the course. Can I still join it?"
query_vector = embedding_model.encode(query)

search_query = {
    "size": 1,  # Nous ne voulons que le document avec le score le plus élevé
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'text_vector') + 1.0",
                "params": {"query_vector": query_vector.tolist()}
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

# Obtenir l'ID du document avec le score le plus élevé
top_hit = response['hits']['hits'][0]
top_id = top_hit['_id']
top_score = top_hit['_score']

print(f"ID du document avec le score le plus élevé : {top_id}")
print(f"Score : {top_score}")

ID du document avec le score le plus élevé : ee58a693
Score : 1.6506574


In [78]:
def elastic_search(query, num_results=5):
    query_vector = embedding_model.encode(query)
    
    search_query = {
        "size": num_results,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'text_vector') + 1.0",
                    "params": {"query_vector": query_vector.tolist()}
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    return [hit['_id'] for hit in response['hits']['hits']]

In [83]:
def calculate_elastic_hitrate(ground_truth, num_results=5):
    hits = 0
    total = len(ground_truth)
    
    for item in ground_truth:
        query = item['question']
        expected_id = item['document']
        
        results = elastic_search(embedding_vector, num_results=num_results)
        
        if expected_id in results:
            hits += 1
    
    return hits / total

In [84]:
calculate_elastic_hitrate(ground_truth,5)

IndexError: invalid index to scalar variable.